In [62]:
import xarray as xa
import numpy

### Daten einlesen

In [3]:
#all:
#mslp_data = xa.open_mfdataset('/raid/home/srvx7/lehre/users/a1401391/JRA-55/mslp_daily/*')
#rh_data = xa.open_mfdataset('/raid/home/srvx7/lehre/users/a1401391/JRA-55/rh_daily/*')
#spfh_data = xa.open_mfdataset('/raid/home/srvx7/lehre/users/a1401391/JRA-55/spfh_daily/*')

#test with subsets:
mslp_data = xa.open_mfdataset('/raid/home/srvx7/lehre/users/a1402239/JRA-55_subsets/mslp/*')
rh_data = xa.open_mfdataset('/raid/home/srvx7/lehre/users/a1402239/JRA-55_subsets/rh/*')
spfh_data = xa.open_mfdataset('/raid/home/srvx7/lehre/users/a1402239/JRA-55_subsets/sh/*')

In [4]:
print (mslp_data)

<xarray.Dataset>
Dimensions:                (g0_lat_1: 29, g0_lon_2: 29, initial_time0_hours: 5844)
Coordinates:
  * g0_lat_1               (g0_lat_1) float32 67.5 66.25 65.0 ... 33.75 32.5
  * g0_lon_2               (g0_lon_2) float32 350.0 351.25 352.5 ... 23.75 25.0
  * initial_time0_hours    (initial_time0_hours) datetime64[ns] 1958-01-01 ... 1961-12-31T18:00:00
Data variables:
    PRMSL_GDS0_MSL         (initial_time0_hours, g0_lat_1, g0_lon_2) float32 dask.array<shape=(5844, 29, 29), chunksize=(1460, 29, 29)>
    initial_time0          (initial_time0_hours) |S18 dask.array<shape=(5844,), chunksize=(1460,)>
    initial_time0_encoded  (initial_time0_hours) float64 dask.array<shape=(5844,), chunksize=(1460,)>
Attributes:
    creation_date:  Tue May 14 05:24:42 MDT 2019
    NCL_Version:    6.6.2
    system:         Linux casper12 3.10.0-693.21.1.el7.x86_64 #1 SMP Wed Mar ...
    Conventions:    None
    grib_source:    anl_surf125.002_prmsl.1958010100_1958123118.salmi370402.grb
    t

### Mittelung

In [45]:
daily_mean_mslp = mslp_data.resample(initial_time0_hours='1D').mean()
daily_mean_rh = rh_data.resample(initial_time0_hours='1D').mean()
daily_mean_spfh = spfh_data.resample(initial_time0_hours='1D').mean()

In [46]:
daily_mean_mslp = daily_mean_mslp.chunk(21) #Chunksize anpasse ???
daily_mean_rh = daily_mean_rh.chunk(21)
daily_mean_spfh = daily_mean_spfh.chunk(21)

In [47]:
print (daily_mean_mslp)

<xarray.Dataset>
Dimensions:                (g0_lat_1: 29, g0_lon_2: 29, initial_time0_hours: 1461)
Coordinates:
  * initial_time0_hours    (initial_time0_hours) datetime64[ns] 1958-01-01 ... 1961-12-31
  * g0_lat_1               (g0_lat_1) float32 67.5 66.25 65.0 ... 33.75 32.5
  * g0_lon_2               (g0_lon_2) float32 350.0 351.25 352.5 ... 23.75 25.0
Data variables:
    PRMSL_GDS0_MSL         (initial_time0_hours, g0_lat_1, g0_lon_2) float32 dask.array<shape=(1461, 29, 29), chunksize=(21, 21, 21)>
    initial_time0_encoded  (initial_time0_hours) float64 dask.array<shape=(1461,), chunksize=(21,)>


In [48]:
# Rollendes Fenster, 1 in Mitte, 10 Tage davor, 10 Tage danach
daily_roll_mslp = daily_mean_mslp.rolling(initial_time0_hours=21,center=1).mean()
daily_roll_rh = daily_mean_rh.rolling(initial_time0_hours=21,center=1).mean()
daily_roll_spfh = daily_mean_spfh.rolling(initial_time0_hours=21,center=1).mean()

In [49]:
daily_mean_mslp = daily_roll_mslp.groupby('initial_time0_hours.dayofyear').mean('initial_time0_hours')
daily_mean_rh = daily_roll_rh.groupby('initial_time0_hours.dayofyear').mean('initial_time0_hours')
daily_mean_spfh = daily_roll_spfh.groupby('initial_time0_hours.dayofyear').mean('initial_time0_hours')

In [61]:
print (daily_mean_mslp)

<xarray.Dataset>
Dimensions:                (dayofyear: 366, g0_lat_1: 29, g0_lon_2: 29)
Coordinates:
  * g0_lat_1               (g0_lat_1) float32 67.5 66.25 65.0 ... 33.75 32.5
  * g0_lon_2               (g0_lon_2) float32 350.0 351.25 352.5 ... 23.75 25.0
  * dayofyear              (dayofyear) int64 1 2 3 4 5 6 ... 362 363 364 365 366
Data variables:
    PRMSL_GDS0_MSL         (dayofyear, g0_lat_1, g0_lon_2) float32 dask.array<shape=(366, 29, 29), chunksize=(1, 21, 21)>
    initial_time0_encoded  (dayofyear) float64 dask.array<shape=(366,), chunksize=(1,)>


### Anomalien

In [56]:
# Anomalie = Tageswerte - Tagesmittelwert
ano_mslp = daily_roll_mslp.groupby('initial_time0_hours.dayofyear') - daily_mean_mslp
ano_rh = daily_roll_rh.groupby('initial_time0_hours.dayofyear') - daily_mean_rh
ano_spfh = daily_roll_spfh.groupby('initial_time0_hours.dayofyear') - daily_mean_spfh
print (ano_mslp)

<xarray.Dataset>
Dimensions:                (g0_lat_1: 29, g0_lon_2: 29, initial_time0_hours: 1461)
Coordinates:
  * g0_lat_1               (g0_lat_1) float32 67.5 66.25 65.0 ... 33.75 32.5
  * g0_lon_2               (g0_lon_2) float32 350.0 351.25 352.5 ... 23.75 25.0
  * initial_time0_hours    (initial_time0_hours) datetime64[ns] 1958-01-01 ... 1961-12-31
    dayofyear              (initial_time0_hours) int64 1 2 3 4 ... 363 364 365
Data variables:
    PRMSL_GDS0_MSL         (initial_time0_hours, g0_lat_1, g0_lon_2) float32 dask.array<shape=(1461, 29, 29), chunksize=(1, 21, 21)>
    initial_time0_encoded  (initial_time0_hours) float64 dask.array<shape=(1461,), chunksize=(1,)>


### Normieren der Anomalien

In [76]:
std_mslp = daily_mean_mslp.groupby('dayofyear').std()
std_rh = daily_mean_rh.groupby('dayofyear').std()
std_spfh = daily_mean_spfh.groupby('dayofyear').std()

/opt/anaconda3/lib/python3.7/site-packages/xarray/core/groupby.py:758: FutureWarning: Default reduction dimension will be changed to the grouped dimension in a future version of xarray. To silence this warning, pass dim=xarray.ALL_DIMS explicitly.
  allow_lazy=True, **kwargs)


In [70]:
print (std_mslp)

<xarray.Dataset>
Dimensions:                (dayofyear: 366)
Coordinates:
  * dayofyear              (dayofyear) int64 1 2 3 4 5 6 ... 362 363 364 365 366
Data variables:
    PRMSL_GDS0_MSL         (dayofyear) float32 dask.array<shape=(366,), chunksize=(1,)>
    initial_time0_encoded  (dayofyear) float64 dask.array<shape=(366,), chunksize=(1,)>


In [77]:
ano_nom_mslp = ano_mslp.groupby('initial_time0_hours.dayofyear')/std_mslp
ano_nom_rh = ano_rh.groupby('initial_time0_hours.dayofyear')/std_rh
ano_nom_spfh = ano_spfh.groupby('initial_time0_hours.dayofyear')/std_spfh

In [75]:
print (ano_nom_mslp)

<xarray.Dataset>
Dimensions:                (g0_lat_1: 29, g0_lon_2: 29, initial_time0_hours: 1461)
Coordinates:
  * g0_lat_1               (g0_lat_1) float32 67.5 66.25 65.0 ... 33.75 32.5
  * g0_lon_2               (g0_lon_2) float32 350.0 351.25 352.5 ... 23.75 25.0
  * initial_time0_hours    (initial_time0_hours) datetime64[ns] 1958-01-01 ... 1961-12-31
    dayofyear              (initial_time0_hours) int64 1 2 3 4 ... 363 364 365
Data variables:
    PRMSL_GDS0_MSL         (initial_time0_hours, g0_lat_1, g0_lon_2) float32 dask.array<shape=(1461, 29, 29), chunksize=(1, 21, 21)>
    initial_time0_encoded  (initial_time0_hours) float64 dask.array<shape=(1461,), chunksize=(1,)>